In [4]:
import os
import pandas as pd
from binance.client import Client
from binance.enums import HistoricalKlinesType

api_key = 'y9m94JvyDsSRyHZUQjNN1lVfDnduNMFyQxz83EW650XKwgbIXxB85xGv7CDQfW8Z'
api_secret = 'cHvzTpnE3Qp468ZcAIadtIQSKvV6HdbXsnwbTAhlTCg9Q0uydcxVKUwP7rqLrcpl'

DATA_PATH = 'data'
klines_column_names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 
                       'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

client = Client(api_key, api_secret, {"timeout": 600})

if not os.path.isdir(DATA_PATH):
    os.mkdir(DATA_PATH)

In [9]:
intervals = {
#              '1day': Client.KLINE_INTERVAL_1DAY,
#              '4hour': Client.KLINE_INTERVAL_4HOUR, 
#              '1hour': Client.KLINE_INTERVAL_1HOUR,
#              '30min': Client.KLINE_INTERVAL_30MINUTE, 
#              '15min': Client.KLINE_INTERVAL_15MINUTE, 
#              '5min': Client.KLINE_INTERVAL_5MINUTE, 
#              '3min': Client.KLINE_INTERVAL_3MINUTE, 
             '1min': Client.KLINE_INTERVAL_1MINUTE
            }

months = ['January', 'February', 'March', 'April', 'May', 'June', 
          'July', 'August', 'September', 'October', 'November', 'December']

pairs = ['btcusdt', 'bnbusdt', 'xrpusdt', 'solusdt', 'bchusdt', 'eosusdt', 'atomusdt']

start_day = 1
start_month = months[6]
start_year = 2022

end_day = 1
end_month = months[7]
end_year = 2022

In [10]:
for pair in pairs:
    if not os.path.isdir(os.path.join(DATA_PATH, str(start_year))):
        os.mkdir(os.path.join(DATA_PATH, str(start_year)))

    if not os.path.isdir(os.path.join(DATA_PATH, str(start_year), start_month.lower())):
        os.mkdir(os.path.join(DATA_PATH, str(start_year), start_month.lower()))

    start_date = '{start_day} {start_month}, {start_year}'.format(start_day=start_day, start_month=start_month, start_year=start_year)
    end_date = '{end_day} {end_month}, {end_year}'.format(end_day=end_day, end_month=end_month, end_year=end_year)

    for i in intervals:
        klines = client.get_historical_klines(symbol = pair.upper(), 
                                     interval = intervals[i], 
                                     start_str = start_date,
                                     end_str = end_date,
                                     klines_type = HistoricalKlinesType.FUTURES)
        df_klines = pd.DataFrame(klines, columns=klines_column_names).set_index('Open time')
        df_klines.index = pd.to_datetime(df_klines.index, unit='ms')
        df_klines = df_klines.astype({'Open':'double', 'High':'double', 'Low':'double', 'Close':'double', 'Volume': 'double'})

        print(df_klines.shape)
        df_klines.to_csv(os.path.join(DATA_PATH,
                                      str(start_year),
                                      start_month.lower(), 
                                      '{pair}.{interval}.{start_date}-{end_date}.csv'.format(pair=pair.lower(), interval=i, 
                                                                                             start_date=str(start_day) + start_month.lower() + str(start_year), 
                                                                                             end_date=str(end_day) + end_month.lower() + str(end_year))))

c:\users\user1\appdata\local\programs\python\python39\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


(44174, 11)
